In [18]:
import os
import time
import random

import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

import pickle

In [19]:
images_dir = './PetImages'
categories = ['Dog', 'Cat']
img_size = 70

# Preprocessing

In [20]:
training_data = []

def create_training_data():    
    for category in categories:
        path = os.path.join(images_dir, category)
        class_num = categories.index(category) # dog == 0, cat == 1
        for img in os.listdir(path):
            try:
                # using grayscale because color probably doesn't help us here
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_img_arr = cv2.resize(img_arr, (img_size, img_size))
                training_data.append([resized_img_arr, class_num])
            except Exception as e:
                print(e)
            
create_training_data()


OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-sn_xpupm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.5.3) C:\Users\runneradmin\

In [21]:
print(len(training_data))

24946


In [22]:
random.shuffle(training_data)
for sample in training_data[:5]:
    print(sample[1])

1
1
1
1
1


In [23]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
# for some reason regular lists dont work so we convert to np arrays
X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)

In [24]:
# pickle dump to save for later use
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [25]:
if (len(tf.config.list_physical_devices('GPU')) == 0):
    print('gpu not detected, training will be slow')

# Model optimization
## Test different dense layers, layer sizes, conv layers

In [26]:
dense_layer_options = [0, 1, 2]
layer_size_options = [32, 64, 128]
conv_layer_options = [1, 2, 3]


for dense_layer in dense_layer_options:
    for layer_size in layer_size_options:
        for conv_layer in conv_layer_options:
            model_name = f"{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
            tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))

            optimized_model = Sequential()
            
            optimized_model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            optimized_model.add(Activation("relu"))
            optimized_model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                optimized_model.add(Conv2D(layer_size, (3,3)))
                optimized_model.add(Activation("relu"))
                optimized_model.add(MaxPooling2D(pool_size=(2,2)))

            optimized_model.add(Flatten())
            
            for l in range(dense_layer):
                optimized_model.add(Dense(layer_size))
                optimized_model.add(Activation("relu"))

            optimized_model.add(Dense(1))
            optimized_model.add(Activation("sigmoid"))

            optimized_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
            optimized_model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

Epoch 1/10
702/702 [==============================] - 5s 4ms/step - loss: 3.6671 - accuracy: 0.5973 - val_loss: 0.7246 - val_accuracy: 0.6056
Epoch 2/10
702/702 [==============================] - 3s 4ms/step - loss: 0.5992 - accuracy: 0.6851 - val_loss: 0.7366 - val_accuracy: 0.6152
Epoch 3/10
702/702 [==============================] - 3s 4ms/step - loss: 0.5124 - accuracy: 0.7444 - val_loss: 0.8362 - val_accuracy: 0.6325
Epoch 4/10
702/702 [==============================] - 3s 4ms/step - loss: 0.4531 - accuracy: 0.7782 - val_loss: 0.8592 - val_accuracy: 0.6405
Epoch 5/10
702/702 [==============================] - 3s 4ms/step - loss: 0.4280 - accuracy: 0.8004 - val_loss: 0.9706 - val_accuracy: 0.6240
Epoch 6/10
702/702 [==============================] - 3s 4ms/step - loss: 0.3950 - accuracy: 0.8164 - val_loss: 1.1005 - val_accuracy: 0.6401
Epoch 7/10
702/702 [==============================] - 3s 4ms/step - loss: 0.3639 - accuracy: 0.8334 - val_loss: 1.1664 - val_accuracy: 0.6293
Epoch 

702/702 [==============================] - 5s 7ms/step - loss: 0.4128 - accuracy: 0.8109 - val_loss: 0.4926 - val_accuracy: 0.7687
Epoch 9/10
702/702 [==============================] - 5s 7ms/step - loss: 0.3863 - accuracy: 0.8246 - val_loss: 0.5454 - val_accuracy: 0.7563
Epoch 10/10
702/702 [==============================] - 5s 7ms/step - loss: 0.3518 - accuracy: 0.8422 - val_loss: 0.5203 - val_accuracy: 0.7860
Epoch 1/10
702/702 [==============================] - 6s 8ms/step - loss: 4.7366 - accuracy: 0.6271 - val_loss: 0.6771 - val_accuracy: 0.6577
Epoch 2/10
702/702 [==============================] - 5s 7ms/step - loss: 0.5262 - accuracy: 0.7440 - val_loss: 0.7852 - val_accuracy: 0.6405
Epoch 3/10
702/702 [==============================] - 5s 7ms/step - loss: 0.4307 - accuracy: 0.8026 - val_loss: 0.7693 - val_accuracy: 0.6762
Epoch 4/10
702/702 [==============================] - 5s 7ms/step - loss: 0.3716 - accuracy: 0.8320 - val_loss: 0.9516 - val_accuracy: 0.6541
Epoch 5/10
702/7

702/702 [==============================] - 4s 5ms/step - loss: 0.4566 - accuracy: 0.7839 - val_loss: 0.4742 - val_accuracy: 0.7884
Epoch 6/10
702/702 [==============================] - 4s 5ms/step - loss: 0.4154 - accuracy: 0.8063 - val_loss: 0.4660 - val_accuracy: 0.7888
Epoch 7/10
702/702 [==============================] - 4s 5ms/step - loss: 0.3839 - accuracy: 0.8270 - val_loss: 0.4848 - val_accuracy: 0.7812
Epoch 8/10
702/702 [==============================] - 4s 5ms/step - loss: 0.3521 - accuracy: 0.8436 - val_loss: 0.4500 - val_accuracy: 0.8080
Epoch 9/10
702/702 [==============================] - 4s 5ms/step - loss: 0.3199 - accuracy: 0.8609 - val_loss: 0.4637 - val_accuracy: 0.8000
Epoch 10/10
702/702 [==============================] - 4s 5ms/step - loss: 0.2891 - accuracy: 0.8741 - val_loss: 0.4824 - val_accuracy: 0.8028
Epoch 1/10
702/702 [==============================] - 5s 7ms/step - loss: 5.8348 - accuracy: 0.6303 - val_loss: 0.6686 - val_accuracy: 0.6986
Epoch 2/10
702/7

702/702 [==============================] - 8s 11ms/step - loss: 0.5606 - accuracy: 0.7120 - val_loss: 0.4990 - val_accuracy: 0.7539
Epoch 3/10
702/702 [==============================] - 8s 11ms/step - loss: 0.4803 - accuracy: 0.7714 - val_loss: 0.4589 - val_accuracy: 0.7880
Epoch 4/10
702/702 [==============================] - 8s 11ms/step - loss: 0.4302 - accuracy: 0.8004 - val_loss: 0.4501 - val_accuracy: 0.7972
Epoch 5/10
702/702 [==============================] - 8s 11ms/step - loss: 0.3864 - accuracy: 0.8254 - val_loss: 0.4255 - val_accuracy: 0.8052
Epoch 6/10
702/702 [==============================] - 8s 11ms/step - loss: 0.3533 - accuracy: 0.8424 - val_loss: 0.4335 - val_accuracy: 0.8024
Epoch 7/10
702/702 [==============================] - 8s 11ms/step - loss: 0.3073 - accuracy: 0.8664 - val_loss: 0.4475 - val_accuracy: 0.8192
Epoch 8/10
702/702 [==============================] - 8s 11ms/step - loss: 0.2663 - accuracy: 0.8859 - val_loss: 0.4547 - val_accuracy: 0.8132
Epoch 9/10

Epoch 10/10
702/702 [==============================] - 5s 7ms/step - loss: 0.1616 - accuracy: 0.9385 - val_loss: 1.5356 - val_accuracy: 0.6577
Epoch 1/10
702/702 [==============================] - 7s 9ms/step - loss: 0.7367 - accuracy: 0.5791 - val_loss: 0.6349 - val_accuracy: 0.6521
Epoch 2/10
702/702 [==============================] - 5s 8ms/step - loss: 0.5864 - accuracy: 0.6929 - val_loss: 0.5536 - val_accuracy: 0.7363
Epoch 3/10
702/702 [==============================] - 5s 8ms/step - loss: 0.5061 - accuracy: 0.7560 - val_loss: 0.4914 - val_accuracy: 0.7559
Epoch 4/10
702/702 [==============================] - 5s 8ms/step - loss: 0.4516 - accuracy: 0.7897 - val_loss: 0.4504 - val_accuracy: 0.7892
Epoch 5/10
702/702 [==============================] - 5s 8ms/step - loss: 0.4079 - accuracy: 0.8137 - val_loss: 0.4353 - val_accuracy: 0.8024
Epoch 6/10
702/702 [==============================] - 5s 8ms/step - loss: 0.3585 - accuracy: 0.8387 - val_loss: 0.5592 - val_accuracy: 0.7575
Epoch

## Result: 3 conv 0 dense 128 layer size had least eval loss

## Try higher layer size options

In [27]:
layer_size_options = [256, 512, 1024]


for layer_size in layer_size_options:
    model_name = f"3-conv-{layer_size}-nodes-0-dense-{int(time.time())}"
    tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))

    optimized_model = Sequential()

    optimized_model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
    optimized_model.add(Activation("relu"))
    optimized_model.add(MaxPooling2D(pool_size=(2,2)))

    optimized_model.add(Conv2D(layer_size, (3,3)))
    optimized_model.add(Activation("relu"))
    optimized_model.add(MaxPooling2D(pool_size=(2,2)))

    optimized_model.add(Conv2D(layer_size, (3,3)))
    optimized_model.add(Activation("relu"))
    optimized_model.add(MaxPooling2D(pool_size=(2,2)))

    optimized_model.add(Flatten())

    optimized_model.add(Dense(1))
    optimized_model.add(Activation("sigmoid"))

    optimized_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    optimized_model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

Epoch 1/10
702/702 [==============================] - 17s 23ms/step - loss: 0.8738 - accuracy: 0.6254 - val_loss: 0.5571 - val_accuracy: 0.7218
Epoch 2/10
702/702 [==============================] - 14s 21ms/step - loss: 0.5423 - accuracy: 0.7283 - val_loss: 0.5072 - val_accuracy: 0.7555
Epoch 3/10
702/702 [==============================] - 14s 21ms/step - loss: 0.4863 - accuracy: 0.7655 - val_loss: 0.4547 - val_accuracy: 0.7908
Epoch 4/10
702/702 [==============================] - 14s 21ms/step - loss: 0.4535 - accuracy: 0.7851 - val_loss: 0.4720 - val_accuracy: 0.7964
Epoch 5/10
702/702 [==============================] - 14s 21ms/step - loss: 0.4175 - accuracy: 0.8090 - val_loss: 0.4253 - val_accuracy: 0.8200
Epoch 6/10
702/702 [==============================] - 14s 21ms/step - loss: 0.3805 - accuracy: 0.8280 - val_loss: 0.4036 - val_accuracy: 0.8253
Epoch 7/10
702/702 [==============================] - 14s 21ms/step - loss: 0.3584 - accuracy: 0.8409 - val_loss: 0.4958 - val_accuracy:

## Final results: 3 conv layers 64 layer size 0 dense

# Final model training

In [28]:
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

X = X / 255.0
tensorboard = TensorBoard(log_dir="logs/{3-conv-64-nodes-0-dense}")
model = Sequential()

model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

model.save('64x3-CNN.h5')

Epoch 1/10
702/702 [==============================] - 7s 9ms/step - loss: 0.6263 - accuracy: 0.6434 - val_loss: 0.5743 - val_accuracy: 0.7134
Epoch 2/10
702/702 [==============================] - 5s 7ms/step - loss: 0.5088 - accuracy: 0.7537 - val_loss: 0.4840 - val_accuracy: 0.7695
Epoch 3/10
702/702 [==============================] - 6s 8ms/step - loss: 0.4511 - accuracy: 0.7897 - val_loss: 0.4383 - val_accuracy: 0.8020
Epoch 4/10
702/702 [==============================] - 6s 8ms/step - loss: 0.4107 - accuracy: 0.8141 - val_loss: 0.4230 - val_accuracy: 0.8116
Epoch 5/10
702/702 [==============================] - 6s 8ms/step - loss: 0.3721 - accuracy: 0.8312 - val_loss: 0.4077 - val_accuracy: 0.8212
Epoch 6/10
702/702 [==============================] - 5s 7ms/step - loss: 0.3472 - accuracy: 0.8453 - val_loss: 0.3681 - val_accuracy: 0.8397
Epoch 7/10
702/702 [==============================] - 5s 7ms/step - loss: 0.3184 - accuracy: 0.8614 - val_loss: 0.3740 - val_accuracy: 0.8377
Epoch 